<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_exotic_energy_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
train_exotic_energy.py

Physics-Informed AI pipeline for predicting optimized negative Casimir energy
as a function of plate distance and permittivity.

Pipeline steps:
 1. Synthetic dataset of (distance, permittivity) → Casimir energy
 2. Physics‐informed loss: supervised MSE + analytic Casimir residual
 3. MLP with LayerNorm & Dropout for uncertainty estimation
 4. MC‐Dropout inference to quantify predictive variance
 5. Training loop with AdamW, ReduceLROnPlateau, early stopping
 6. Visualizations: loss curves, true vs. predicted scatter
"""

import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Dataset
# ------------------------------------------------------------------------------
def casimir_effect(d, eps):
    """Analytic Casimir energy between parallel plates."""
    return -np.pi**2 / (240 * d**4) * eps

class CasimirDataset(Dataset):
    def __init__(self, n_samples=5000, seed=42):
        np.random.seed(seed)
        d   = np.random.uniform(0.1, 1.0, size=(n_samples,1)).astype(np.float32)
        eps = np.random.uniform(1.0, 10.0, size=(n_samples,1)).astype(np.float32)
        X_raw = np.hstack([d, eps])

        Y_raw = casimir_effect(d, eps)
        Y_raw += 0.01 * np.random.randn(*Y_raw.shape).astype(np.float32)

        # Compute normalization statistics (NumPy)
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0)
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0)

        # Normalize (NumPy)
        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx])
        y = torch.from_numpy(self.Y[idx])
        return x, y

# ------------------------------------------------------------------------------
# 2. Model Architecture
# ------------------------------------------------------------------------------
class ExoticEnergyAI(nn.Module):
    def __init__(self, input_dim=2, hidden_dims=(64,64), output_dim=1, p_drop=0.1):
        super().__init__()
        layers = []
        dim = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics‐Informed Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, inp, stats):
    """
    Enforce analytic Casimir relation:
      E_true = -π² / (240 · d⁴) · ε
    All operations in torch to avoid dtype mismatches.
    """
    # Denormalize inputs (torch)
    d   = inp[:, 0] * stats['X_std'][0] + stats['X_mean'][0]
    eps = inp[:, 1] * stats['X_std'][1] + stats['X_mean'][1]

    # True Casimir energy (torch)
    E_true = - (math.pi ** 2) / 240.0 * eps / (d ** 4)

    # Denormalize prediction
    E_pred = pred.squeeze() * stats['Y_std'] + stats['Y_mean']

    return nn.MSELoss()(E_pred, E_true)

def total_loss(pred, true, inp, stats, lambda_phys=1.0):
    mse_loss  = nn.MSELoss()(pred, true)
    phys_loss = physics_residual(pred, inp, stats)
    return mse_loss + lambda_phys * phys_loss, mse_loss, phys_loss

# ------------------------------------------------------------------------------
# 4. MC‐Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, n_samples=100):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            preds.append(model(x).cpu().numpy())
    arr = np.stack(preds, axis=0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-3, wd=1e-5, lambda_phys=1.0,
          max_epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, max_epochs + 1):
        # Training
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, lambda_phys)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, lambda_phys)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Early stopping
        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_exotic_energy_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_exotic_energy_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_losses(history):
    plt.figure()
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'],   label='Val')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training Curve")
    plt.tight_layout()
    plt.show()

def plot_scatter(y_true, y_pred):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel("True Energy")
    plt.ylabel("Pred Energy")
    plt.title("True vs Predicted Casimir Energy")
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset and normalization stats
    ds = CasimirDataset(n_samples=8000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std':  torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std':  torch.tensor(ds.Y_std,  device=device),
    }

    # Train/validation split
    n_val = int(0.2 * len(ds))
    n_trn = len(ds) - n_val
    trn_ds, val_ds = random_split(ds, [n_trn, n_val])
    trn_loader = DataLoader(trn_ds, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=256)

    # Initialize and train model
    model   = ExoticEnergyAI().to(device)
    history = train(model, trn_loader, val_loader, stats, device)

    # Plot training history
    plot_losses(history)

    # Evaluate on full dataset
    X_all = torch.from_numpy(ds.X).to(device).float()
    with torch.no_grad():
        y_pred_norm = model(X_all).cpu().numpy().squeeze()

    # Denormalize for final scatter
    y_true = (ds.Y * ds.Y_std + ds.Y_mean).squeeze()
    y_pred = y_pred_norm * ds.Y_std + ds.Y_mean
    plot_scatter(y_true, y_pred)